# DataJoint Workflow Guide for Creating a New Ephys Experiment



This notebook provides a walkthrough for adding a new ephys experiment to the pipeline. **This notebook is optional and only needed if the team decides to create new experiments and conditions.**


To add a new `Experiment` entry, we recommend using the **SciViz website** designed for this purpose. Please refer to the [SciViz section](../README.md#sciviz-website).

**_Note:_**

- The examples in this notebook use a sample dataset. Replace these entries with your actual database entries to access and analyze your data.


### **Key Steps**


- **Setup**

- **Step 1: Add a New Experiment in the `Experiment` Table**

- **Step 2: Add the Corresponding `ExperimentDirectory` Entry**

#### **Setup**


First, import the necessary packages for the data pipeline and essential schemas.


In [1]:
## Ensure the correct working directory

import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
from datetime import datetime

In [3]:
from workflow.pipeline import culture

[2024-07-23 18:21:34,455][INFO]: Connecting milagros@db.datajoint.com:3306
[2024-07-23 18:21:36,052][INFO]: Connected milagros@db.datajoint.com:3306


#### **Step 1: Add a New Experiment in the `Experiment` Table (Optional)**

The `Experiment` table contains the experiment to be performed on each organoid under different drug conditions. Example:

In [4]:
(culture.Experiment & 'organoid_id="O09"').proj(
    "experiment_end_time", "drug_name", "drug_concentration", "experiment_plan"
)

organoid_id e.g. O17,experiment_start_time,experiment_end_time,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
O09,2023-05-03 17:33:00,2023-05-18 12:15:00,Control,nan,ephys
O09,2023-05-18 12:25:00,2023-05-18 18:15:00,4-AP,100.0,ephys
O09,2023-05-18 18:15:00,2023-05-19 09:30:00,No Drug,nan,ephys
O09,2023-05-19 09:30:00,2023-05-19 15:35:00,Bicuculline,50.0,ephys
O09,2023-05-19 15:45:00,2023-05-20 15:40:00,Tetrodotoxin,1.0,ephys


To add a new `Experiment` entry, we recommend using the **SciViz website** designed for this purpose. Please refer to the [SciViz section](../README.md#sciviz-website). Alternatively, you can always insert it using `insert1` or `insert`, as demonstrated in other CREATE notebooks.

#### **Step 2: Add the Corresponding `ExperimentDirectory` Entry (Optional)**

If you have already inserted a new `Experiment` entry, please ensure that the new entry is also included in the `ExperimentDirectory` table.

The `ExperimentDirectory` table contains the specific relative path to the subject data directory for the experiment entries. Example:

In [5]:
(culture.ExperimentDirectory & 'organoid_id="O09"')

organoid_id e.g. O17,experiment_start_time,experiment_directory Path to the subject data directory for long term recordings
O09,2023-05-03 17:33:00,O09-12_raw
O09,2023-05-18 12:25:00,O09-12_raw
O09,2023-05-18 18:15:00,O09-12_raw
O09,2023-05-19 09:30:00,O09-12_raw
O09,2023-05-19 15:45:00,O09-12_raw


The following code cell is designed to update an `ExperimentDirectory` table by associating new experiment entries with their corresponding batch directories. 

In [6]:
# Fetch experiment keys not present in the ExperimentDirectory table
experiment_keys = (culture.Experiment - culture.ExperimentDirectory).fetch("KEY")

# Define batches and their corresponding parent directories
batches = {
    "batch1": {"ids": ("O09", "O10", "O11", "O12"), "parent": "O09-12_raw"},
    "batch2": {"ids": ("O13", "O14", "O15", "O16"), "parent": "O13-16_raw"},
    "batch3": {"ids": ("O17", "O18", "O19", "O20"), "parent": "O17-20_raw"},
}

# Create string representations of batches
for batch in batches.values():
    batch["ids_str"] = f"({', '.join(repr(x) for x in batch['ids'])})"


# Function to find the parent directory for a given key
def find_parent_directory(key):
    for batch in batches.values():
        if key in (culture.Experiment & f"organoid_id IN {batch['ids_str']}"):
            return batch["parent"]
    return None


# Insert experiment keys into ExperimentDirectory with corresponding parent directories
for key in experiment_keys:
    parent_directory = find_parent_directory(key)
    if parent_directory:
        exp_dir = dict(key, experiment_directory=parent_directory)
        culture.ExperimentDirectory.insert1(exp_dir, skip_duplicates=True)

In [7]:
# Confirm that all the entries are present in both tables (empty table means all entries are present)
culture.Experiment - culture.ExperimentDirectory

organoid_id e.g. O17,experiment_start_time,experiment_end_time,user,lineage_id de-identified code (e.g. hmau001),induction_culture_date,induction_culture_plate,post_induction_culture_date,post_induction_culture_plate,isolated_rosette_culture_date Date for isolating the rosette,isolated_rosette_culture_plate,organoid_culture_date,organoid_culture_plate,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
